In [1]:
import pandas as pd
import numpy as np

**Загрузка данных**

In [2]:
validate_train_url = "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/DZW9I4MwAJrl_A"
validate_test_url = "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/8nSFbNQY92HCng"
validate_answers_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/UsSATGKzLrhBFQ'
validate_answers_test = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/iu8jIJk1C15mww'
history_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/BkZWvVVDjfB1rw'
users_train_url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/SEz-05NG0vpkKA'
validate_train = pd.read_csv(validate_train_url, sep='\t')
validate_test = pd.read_csv(validate_test_url, sep='\t')
validate_answers_train = pd.read_csv(validate_answers_train_url, sep='\t')
validate_answers_test = pd.read_csv(validate_answers_test, sep='\t')
history_train = pd.read_csv(history_train_url, sep='\t')
users_train = pd.read_csv(users_train_url, sep='\t')

In [3]:
validate_train = validate_train.drop('Unnamed: 0', axis=1)
validate_test = validate_test.drop('Unnamed: 0', axis=1)
validate_answers_train = validate_answers_train.drop('Unnamed: 0', axis=1)
validate_answers_test = validate_answers_test.drop('Unnamed: 0', axis=1)
history_train = history_train.drop('Unnamed: 0', axis=1)
users_train = users_train.drop('Unnamed: 0', axis=1)

**Предподсчет**

In [4]:
views_for_users_by_publishers = {}
for user_id in users_train['user_id'].unique():
  temp = history_train[history_train['user_id'] == user_id]
  publishers = temp['publisher'].tolist()
  views_for_users_by_publishers[user_id] = []
  for i in range(22):
    views_for_users_by_publishers[user_id].append(publishers.count(i))

In [6]:
users_statistics_by_publishers = {}
for user_id in list(users_train['user_id'].unique()):
    user_history = history_train[history_train['user_id'] == user_id]
    for i in range(1,22):
        temp_user_history = user_history[user_history['publisher'] == i]
        users_statistics_by_publishers[(user_id, i, 'total_views')] = temp_user_history.shape[0]
        users_statistics_by_publishers[(user_id, i, 'views_share')] = temp_user_history.shape[0]/user_history.shape[0] if user_history.shape[0] != 0 else 0
        users_statistics_by_publishers[(user_id, i, 'min_cpm')] = temp_user_history['cpm'].min()
        users_statistics_by_publishers[(user_id, i, 'max_cpm')] = temp_user_history['cpm'].max()
        users_statistics_by_publishers[(user_id, i, 'mean_cpm')] = temp_user_history['cpm'].mean()
        users_statistics_by_publishers[(user_id, i, 'median_cpm')] = temp_user_history['cpm'].median()
        users_statistics_by_publishers[(user_id, i, 'mean_views_per_day')] = temp_user_history.shape[0]/((temp_user_history['hour'].max()-temp_user_history['hour'].min())/24) if temp_user_history['hour'].max()-temp_user_history['hour'].min() != 0 else 0

**Создадим датасет, чтобы предсказывать вероятность для конкретного пользователя**

In [63]:
def count_views_by_user(user, publishers, views_for_users_by_publishers):
    sum = 0
    for publisher in publishers:
        sum += views_for_users_by_publishers[user][publisher]
    return sum

In [64]:
def make_df(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    users_id = []
    publishers = []
    cpm = []
    one_view_probability = []
    two_view_probability = []
    three_view_probability = []
    views_count_by_publisher = []
    duration = []
    total_views = []
    views_share = []
    min_cpm = []
    max_cpm = []
    mean_cpm = []
    median_cpm = []
    mean_views_per_day = []
    for index in range(validate_train.shape[0]):
        validate_row = validate_train.iloc[index]
        validate_answer_row = validate_answers_train.iloc[index]        
        for user_id in list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
            sum_views = count_views_by_user(user_id, list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split())), views_for_users_by_publishers)   
            for publisher_id in list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
                users_id.append(user_id)
                publishers.append(publisher_id)
                duration.append(validate_row['hour_end']-validate_row['hour_start'])
                cpm.append(validate_row['cpm'])
                coefficient = views_for_users_by_publishers[user_id][publisher_id] / sum_views if sum_views != 0 else 0
                one_view_probability.append(validate_answer_row['at_least_one']*coefficient)
                two_view_probability.append(validate_answer_row['at_least_two']*coefficient)
                three_view_probability.append(validate_answer_row['at_least_three']*coefficient)
                total_views.append(users_statistics_by_publishers[(user_id, publisher_id, 'total_views')])
                views_share.append(users_statistics_by_publishers[(user_id, publisher_id, 'views_share')])
                min_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'min_cpm')])
                max_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'max_cpm')])
                mean_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_cpm')])
                median_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'median_cpm')])
                mean_views_per_day.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_views_per_day')])

    return pd.DataFrame({
        "user_id": users_id,
        "publisher": publishers,
        "cpm": cpm,
        "one_view_probability": one_view_probability,
        "two_view_probability": two_view_probability,
        "three_view_probability": three_view_probability,
        'total_view': total_views,
        'view_share': views_share,
        'min_cpm_per_publisher': min_cpm,
        'max_cpm_per_publisher': max_cpm,
        'mean_cpm_per_publisher': mean_cpm,
        'median_cpm_per_publisher': median_cpm,
        'mean_views_per_day': mean_views_per_day,
        'duration': duration
    })

In [66]:
import random
from sklearn.model_selection import train_test_split

random.seed(42)

def preparing_data():
    data = make_df(validate_train, validate_answers_train, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
    data = pd.merge(data, users_train, on='user_id', how='inner')
    data = data.reindex(random.shuffle([i for i in range(data.shape[0])])).reset_index(drop=True)
    data.fillna(0, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['one_view_probability', 'two_view_probability', 'three_view_probability']), data[['one_view_probability', 'two_view_probability', 'three_view_probability']], test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [67]:
X_train, X_test, y_train, y_test = preparing_data()

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, PoissonRegressor, TweedieRegressor, Lasso, Ridge
from sklearn.metrics import r2_score
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', None)
])

param_grid = [
    {
        'model': [PoissonRegressor()],
        'model__alpha': [0, 0.1, 1, 10],
        'model__max_iter': [100, 500, 1000]
    },
    {
        'model': [Lasso(random_state=42)],
        'model__alpha': [0.001, 0.01, 0.1, 1, 10],
        'model__max_iter': [1000, 5000]
    },
    {
        'model': [Ridge(random_state=42)],
        'model__alpha': [0.001, 0.01, 0.1, 1, 10],
        'model__max_iter': [1000, 5000]
    }
]

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=2,
    error_score='raise'
)

In [84]:
def R2_metrix(model_for_one, model_for_two, model_for_three):
    print("Model for one train R2: ", r2_score(y_train['one_view_probability'], model_for_one.predict(X_train)))
    print("Model for one test R2: ", r2_score(y_test['one_view_probability'], model_for_one.predict(X_test)))
    print("Model for two train R2: ", r2_score(y_train['two_view_probability'], model_for_two.predict(X_train)))
    print("Model for two test R2: ", r2_score(y_test['two_view_probability'], model_for_two.predict(X_test)))
    print("Model for three train R2: ", r2_score(y_train['three_view_probability'], model_for_three.predict(X_train)))
    print("Model for three test R2: ", r2_score(y_test['three_view_probability'], model_for_three.predict(X_test)))

In [ ]:
grid_search.fit(X_train, y_train['one_view_probability'])
model_for_one = grid_search.best_estimator_
grid_search.fit(X_train, y_train['two_view_probability'])
model_for_two = grid_search.best_estimator_
grid_search.fit(X_train, y_train['three_view_probability'])
model_for_three = grid_search.best_estimator_

In [72]:
R2_metrix(model_for_one, model_for_two, model_for_three)

Model for one train R2:  0.4810419322136207
Model for one test R2:  0.5042095915097959
Model for two train R2:  0.3226929647105399
Model for two test R2:  0.3152654161548495
Model for three train R2:  0.22326627506727692
Model for three test R2:  0.21848791459216133


In [86]:
def count(ans):
    sums = ans.groupby("user_id")["one"].sum()
    one = sums.mean()
    sums = ans.groupby("user_id")["two"].sum()
    two = sums.mean()
    sums = ans.groupby("user_id")["three"].sum()
    three = sums.mean()

In [90]:
def predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    at_least_one = []
    at_least_two = []
    at_least_three = []
    for i in range(validate_test.shape[0]):
        users_id = []
        publishers = []
        cpm = []
        views_count_by_publisher = []
        duration = []
        total_views = []
        views_share = []
        min_cpm = []
        max_cpm = []
        mean_cpm = []
        median_cpm = []
        mean_views_per_day = []

        validate_row = validate_test.iloc[i]
        for user_id in list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
            sum_views = count_views_by_user(user_id, list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split())), views_for_users_by_publishers)   
            for publisher_id in list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
                users_id.append(user_id)
                publishers.append(publisher_id)
                duration.append(validate_row['hour_end']-validate_row['hour_start'])
                cpm.append(validate_row['cpm'])
                total_views.append(users_statistics_by_publishers[(user_id, publisher_id, 'total_views')])
                views_share.append(users_statistics_by_publishers[(user_id, publisher_id, 'views_share')])
                min_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'min_cpm')])
                max_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'max_cpm')])
                mean_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_cpm')])
                median_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'median_cpm')])
                mean_views_per_day.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_views_per_day')])
        data = pd.DataFrame({
            "user_id": users_id,
            "publisher": publishers,
            "cpm": cpm,
            'total_view': total_views,
            'view_share': views_share,
            'min_cpm_per_publisher': min_cpm,
            'max_cpm_per_publisher': max_cpm,
            'mean_cpm_per_publisher': mean_cpm,
            'median_cpm_per_publisher': median_cpm,
            'mean_views_per_day': mean_views_per_day,
            'duration': duration
            })
        data = pd.merge(data, users_train, on='user_id', how='inner')
        data.fillna(0, inplace=True)

        one = model_for_one.predict(data)
        two = model_for_two.predict(data)
        three = model_for_three.predict(data)
        ans = pd.DataFrame({
            "user_id": users_id,
            "one": one,
            "two": two,
            "three": three
        }) 
        one, two, three = count(ans)
        at_least_one.append(one)
        at_least_two.append(two)
        at_least_three.append(three)
    return pd.DataFrame({
        "at_least_one": at_least_one,
        "at_least_two": at_least_two,
        'at_least_three': at_least_three
    })


    
    

In [74]:
def load_answers(answers_filename):
    return pd.read_csv(answers_filename, sep="\t")


def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

In [75]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)

In [76]:
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 146.88%


Мы получили некий результат, теперь попробуем обучить более сложные модели и оценим его.

In [77]:
from xgboost import XGBRegressor
import xgboost as xgb

print("Версия XGBoost:", xgb.__version__)

!nvidia-smi -L

xgb_params = XGBRegressor(tree_method='gpu_hist').get_params()
print("Поддержка GPU в XGBoost:", xgb_params["tree_method"] == 'gpu_hist')

Версия XGBoost: 2.0.3
GPU 0: Tesla T4 (UUID: GPU-1dc8717b-8a07-aa3d-9513-9a8e930f50fc)
GPU 1: Tesla T4 (UUID: GPU-3294d9ac-e231-0d21-c859-5fab85b35bdd)
Поддержка GPU в XGBoost: True


In [78]:
param_grid = [{
    'model': [XGBRegressor(
        random_state=42,
        device='cuda',
        tree_method='hist',
        eval_metric='rmse'
    )],
    'model__n_estimators':     [50, 100],    
    'model__max_depth':        [7, 9],           
    'model__learning_rate':    [0.1],            
    'model__subsample':        [0.8],            
    'model__colsample_bytree': [0.8],            
    'model__gamma':            [0, 0.1],         
    'model__min_child_weight': [1, 3] 
}]

In [79]:
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=2,
    error_score='raise'
)

In [80]:
grid_search.fit(X_train, y_train['one_view_probability'])
model_for_one = grid_search.best_estimator_
grid_search.fit(X_train, y_train['two_view_probability'])
model_for_two = grid_search.best_estimator_
grid_search.fit(X_train, y_train['three_view_probability'])
model_for_three = grid_search.best_estimator_

In [85]:
R2_metrix(model_for_one, model_for_two, model_for_three)

Model for one train R2:  0.9952362508218107
Model for one test R2:  0.9803142293254508
Model for two train R2:  0.9971556208416549
Model for two test R2:  0.9835618040042433
Model for three train R2:  0.9934730131972286
Model for three test R2:  0.9840733376077702


In [83]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 113.12%


Мы получили неплохие результаты. Попробуем улучшить.

In [89]:
def count(ans):
    sums = ans.groupby("user_id")["one"].sum()
    one = sums.median()
    sums = ans.groupby("user_id")["two"].sum()
    two = sums.median()
    sums = ans.groupby("user_id")["three"].sum()
    three = sums.median()
    return one, two, three

In [91]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 163.61%


Медиана работает хуже.

In [108]:
def count(ans, views_for_users_by_publishers, publisher_ids, user_ids):
    total_views = 0
    users_views = {}
    for user_id in user_ids:
        temp = 0
        for publisher_id in publisher_ids:
            temp+=views_for_users_by_publishers[user_id][publisher_id]
        users_views[user_id]=temp
        total_views+=temp
    
    sums = ans.groupby("user_id", as_index=False)["one"].sum()
    one = 0
    for i in range(sums.shape[0]):
        row = sums.iloc[i]
        one+=row['one']*users_views[row['user_id']]

    sums = ans.groupby("user_id", as_index=False)["two"].sum()
    two = 0
    for i in range(sums.shape[0]):
        row = sums.iloc[i]
        two+=row['two']*users_views[row['user_id']]
        
    sums = ans.groupby("user_id", as_index=False)["three"].sum()
    three = 0
    for i in range(sums.shape[0]):
        row = sums.iloc[i]
        three+=row['three']*users_views[row['user_id']]
        
    return one/total_views, two/total_views, three/total_views

In [107]:
def predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers):
    at_least_one = []
    at_least_two = []
    at_least_three = []
    for i in range(validate_test.shape[0]):
        users_id = []
        publishers = []
        cpm = []
        views_count_by_publisher = []
        duration = []
        total_views = []
        views_share = []
        min_cpm = []
        max_cpm = []
        mean_cpm = []
        median_cpm = []
        mean_views_per_day = []

        validate_row = validate_test.iloc[i]
        for user_id in list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split())):
            publisher_ids = list(map(int, (validate_row['publishers'].replace('(', '').replace(')', '').replace(',', ' ')).split()))
            sum_views = count_views_by_user(user_id, publisher_ids, views_for_users_by_publishers)  
            for publisher_id in publisher_ids:
                users_id.append(user_id)
                publishers.append(publisher_id)
                duration.append(validate_row['hour_end']-validate_row['hour_start'])
                cpm.append(validate_row['cpm'])
                total_views.append(users_statistics_by_publishers[(user_id, publisher_id, 'total_views')])
                views_share.append(users_statistics_by_publishers[(user_id, publisher_id, 'views_share')])
                min_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'min_cpm')])
                max_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'max_cpm')])
                mean_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_cpm')])
                median_cpm.append(users_statistics_by_publishers[(user_id, publisher_id, 'median_cpm')])
                mean_views_per_day.append(users_statistics_by_publishers[(user_id, publisher_id, 'mean_views_per_day')])
        data = pd.DataFrame({
            "user_id": users_id,
            "publisher": publishers,
            "cpm": cpm,
            'total_view': total_views,
            'view_share': views_share,
            'min_cpm_per_publisher': min_cpm,
            'max_cpm_per_publisher': max_cpm,
            'mean_cpm_per_publisher': mean_cpm,
            'median_cpm_per_publisher': median_cpm,
            'mean_views_per_day': mean_views_per_day,
            'duration': duration
            })
        data = pd.merge(data, users_train, on='user_id', how='inner')
        data.fillna(0, inplace=True)

        one = model_for_one.predict(data)
        two = model_for_two.predict(data)
        three = model_for_three.predict(data)
        ans = pd.DataFrame({
            "user_id": users_id,
            "one": one,
            "two": two,
            "three": three
        }) 
        one, two, three = count(ans, views_for_users_by_publishers, publisher_ids, list(map(int, (validate_row['user_ids'].replace('(', '').replace(')', '').replace(',', ' ')).split())))
        at_least_one.append(one)
        at_least_two.append(two)
        at_least_three.append(three)
    return pd.DataFrame({
        "at_least_one": at_least_one,
        "at_least_two": at_least_two,
        'at_least_three': at_least_three
    })


    
    

In [109]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 114.29%


In [111]:
print((pred['at_least_one'] > validate_answers_test['at_least_one']).mean(), (pred['at_least_two'] > validate_answers_test['at_least_two']).mean(), (pred['at_least_three'] > validate_answers_test['at_least_three']).mean())

0.520949720670391 0.6089385474860335 0.5698324022346368


Мы чаще ошибаемся в верх.

In [152]:
from sklearn.metrics import make_scorer

def asymmetric_mse(y_true, y_pred, alpha=2.0):
    error = y_pred - y_true
    mask = error > 0 
    return np.mean(np.where(mask, alpha * error**2, (error**2)/alpha))

asymmetric_scorer = make_scorer(asymmetric_mse, alpha=2.0, greater_is_better=False)

In [153]:

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=asymmetric_scorer,  
    cv=5,
    n_jobs=-1,
    verbose=2,
    error_score='raise'
)

In [ ]:
grid_search.fit(X_train, y_train['one_view_probability'])
model_for_one = grid_search.best_estimator_
grid_search.fit(X_train, y_train['two_view_probability'])
model_for_two = grid_search.best_estimator_
grid_search.fit(X_train, y_train['three_view_probability'])
model_for_three = grid_search.best_estimator_

In [155]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 119.11%


In [156]:
def geometric_relative_error(y_true, y_pred, epsilon=1e-6):
    ratio = (y_pred + epsilon) / (y_true + epsilon)
    log_ratio = np.log(ratio)
    mean_log = np.mean(log_ratio)
    return np.abs(np.exp(mean_log) - 1)  

In [157]:
geometric_scorer = make_scorer(
    geometric_relative_error, 
    epsilon=1e-6,  
    greater_is_better=False 
)

In [158]:
grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=geometric_scorer,  
    cv=5,
    n_jobs=-1,
    verbose=2,
    error_score='raise'
)

In [ ]:
grid_search.fit(X_train, y_train['one_view_probability'])
model_for_one = grid_search.best_estimator_
grid_search.fit(X_train, y_train['two_view_probability'])
model_for_two = grid_search.best_estimator_
grid_search.fit(X_train, y_train['three_view_probability'])
model_for_three = grid_search.best_estimator_

In [160]:
pred = predict(model_for_one, model_for_two, model_for_three, validate_test, views_for_users_by_publishers, history_train, users_train, users_statistics_by_publishers)
percentage_error_test = get_smoothed_mean_log_accuracy_ratio(validate_answers_test, pred)
print(f'Percentage Error on Test: {percentage_error_test}%')

Percentage Error on Test: 115.96%
